In [1]:
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras    
from tensorflow.keras.layers import Flatten   
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers
from keras.optimizers import Adam, Adamax, Adadelta, Nadam, SGD
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


drive.mount("/content/drive")
path = "/content/drive/MyDrive/Capstone/exercise_datasetV2.csv"
df = pd.read_csv(path)
print(df.head())
banyak_kategori = len(df.index)


Mounted at /content/drive
  Activity, Exercise or Sport (1 hour) Intensity Description  \
0          Cycling, mountain bike, bmx                   NaN   
1  Cycling, <10 mph, leisure bicycling                   NaN   
2             Cycling, >20 mph, racing                   NaN   
3          Cycling, 10-11.9 mph, light                   NaN   
4       Cycling, 12-13.9 mph, moderate                   NaN   

   Duration (minutes)  Calories per kg  
0                  60         1.750730  
1                  60         0.823236  
2                  60         3.294974  
3                  60         1.234853  
4                  60         1.647825  


In [2]:
list_berat = []
for i in range(len(df.index)):
  list_berat.append(1)

df['berat'] =  list_berat
dict_df = {'Activity, Exercise or Sport (1 hour)' : [], 'Duration (minutes)': [], 'Calories per kg': [], 'berat' : []}
df_new = df
for index, row in df.iterrows():
  print(index)
  menit = row['Duration (minutes)']
  activity = row['Activity, Exercise or Sport (1 hour)']
  calories = row['Calories per kg']
  for i in range(1,menit):
    for j in range(2,101):
      new_calories = calories*1.0/60*i*j
      list_activity = dict_df.get('Activity, Exercise or Sport (1 hour)')
      list_duration = dict_df.get('Duration (minutes)')
      list_calories = dict_df.get('Calories per kg')
      list_berat = dict_df.get('berat')
      list_activity.append(activity)
      list_duration.append(i)
      list_calories.append(new_calories)
      list_berat.append(j)
      #new_row = pd.DataFrame({'Activity, Exercise or Sport (1 hour)' : [activity], 'Duration (minutes)': [i], 'Calories per kg': [new_calories], 'berat' : [j]})
      
df_curr = pd.DataFrame(dict_df)   
df_new = pd.concat([df_curr, df_new.loc[:]]).reset_index(drop=True)
#df2 = pd.concat([new_row,df.loc[:]]).reset_index(drop=True)
print(df_new.head())
print(df_new.tail())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
  Activity, Exercise or Sport (1 hour)  Duration (minutes)  Calories per kg  \
0          Cycling, mountain bike, bmx 

In [3]:
print(len(df_new.index))
print(df_new.describe())
print(df_new.dtypes)
df_new.rename(columns = {'Activity, Exercise or Sport (1 hour)':'activity', 'Duration (minutes)' : 'durasi' , 'Calories per kg' : 'calories' }, inplace = True)
print(df_new.head())

1448816
       Duration (minutes)  Calories per kg         berat
count        1.448816e+06     1.448816e+06  1.448816e+06
mean         3.000514e+01     3.467251e+01  5.099144e+01
std          1.703246e+01     3.748635e+01  2.858243e+01
min          1.000000e+00     1.033558e-02  1.000000e+00
25%          1.500000e+01     8.237434e+00  2.600000e+01
50%          3.000000e+01     2.219663e+01  5.100000e+01
75%          4.500000e+01     4.774767e+01  7.600000e+01
max          6.000000e+01     3.644815e+02  1.000000e+02
Activity, Exercise or Sport (1 hour)     object
Duration (minutes)                        int64
Calories per kg                         float64
berat                                     int64
Intensity Description                    object
dtype: object
                      activity  durasi  calories  berat Intensity Description
0  Cycling, mountain bike, bmx       1  0.058358      2                   NaN
1  Cycling, mountain bike, bmx       1  0.087536      3              

In [4]:
target = df['Activity, Exercise or Sport (1 hour)']
print(df_new.head())
numeric_feature_names = ['durasi', 'calories', 'berat']
numeric_features = df_new[numeric_feature_names]
numeric_features.head()

                      activity  durasi  calories  berat Intensity Description
0  Cycling, mountain bike, bmx       1  0.058358      2                   NaN
1  Cycling, mountain bike, bmx       1  0.087536      3                   NaN
2  Cycling, mountain bike, bmx       1  0.116715      4                   NaN
3  Cycling, mountain bike, bmx       1  0.145894      5                   NaN
4  Cycling, mountain bike, bmx       1  0.175073      6                   NaN


,durasi,calories,berat
0,1,0.058358,2
1,1,0.087536,3
2,1,0.116715,4
3,1,0.145894,5
4,1,0.175073,6


In [6]:
jumlah_class = len(df_new['activity'].value_counts())
print(jumlah_class)

248


In [7]:
df_new['activity'] = df_new['activity'].astype('category')
df_new['activity_category'] = df_new['activity'].cat.codes.astype('category')
print(df_new.head())

                      activity  durasi  calories  berat Intensity Description  \
0  Cycling, mountain bike, bmx       1  0.058358      2                   NaN   
1  Cycling, mountain bike, bmx       1  0.087536      3                   NaN   
2  Cycling, mountain bike, bmx       1  0.116715      4                   NaN   
3  Cycling, mountain bike, bmx       1  0.145894      5                   NaN   
4  Cycling, mountain bike, bmx       1  0.175073      6                   NaN   

  activity_category  
0                61  
1                61  
2                61  
3                61  
4                61  


In [8]:
df_new_2 = df_new.drop(columns = ['activity', 'Intensity Description', 'activity_category'])
sc = StandardScaler()
x = pd.DataFrame(sc.fit_transform(df_new_2))

In [9]:
df_new_2['durasi'] = MinMaxScaler().fit_transform(np.array(df_new_2['durasi']).reshape(-1,1))
df_new_2['calories'] = MinMaxScaler().fit_transform(np.array(df_new_2['calories']).reshape(-1,1))
df_new_2['berat'] = MinMaxScaler().fit_transform(np.array(df_new_2['berat']).reshape(-1,1))

In [10]:
y = tf.keras.utils.to_categorical(df_new["activity_category"].values, num_classes=jumlah_class)

x_train, x_test, y_train, y_test = train_test_split(x.values, y, test_size=0.2)

In [11]:
print(x_train)
print(y_train)
print(x_test)
print(y_test)

[[-0.64612751 -0.21978469  1.36477452]
 [ 0.23454433  1.76455689  0.73501679]
 [-1.35066498 -0.85191691 -1.11926986]
 ...
 [-1.29195353 -0.81506008 -1.25921602]
 [-0.58741606 -0.10019745  1.71463993]
 [ 0.9390818   1.096578   -0.38455251]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[-0.17643586 -0.81368662 -1.46913527]
 [ 1.05650471 -0.5381065  -1.39916218]
 [-0.82226188 -0.24853981  0.9099495 ]
 ...
 [ 0.99779325 -0.29243914 -1.04929678]
 [-0.99839625 -0.1393166   0.52509755]
 [ 0.58681306  1.35641312  1.32978798]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [12]:
from keras.engine import sequential
def get_model():
    model =  tf.keras.Sequential([
        Dense(1024, activation='relu'),
        
        Dense(512, activation='relu'),
        
        
        
       
        
        
        
        Dense(banyak_kategori, activation='softmax')
    ])

    optimizer = SGD(learning_rate=0.01)

    model.compile(optimizer= optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [13]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

In [14]:
model = get_model()

In [ ]:

model_fit = model.fit(x_train, 
                      y_train, 
                      epochs = 100,
                      validation_data = (x_test, y_test))

Epoch 1/100
36221/36221 [==============================] - 565s 16ms/step - loss: 4.7259 - accuracy: 0.0239 - val_loss: 4.1264 - val_accuracy: 0.0367
Epoch 2/100
36221/36221 [==============================] - 540s 15ms/step - loss: 3.8702 - accuracy: 0.0550 - val_loss: 4.1393 - val_accuracy: 0.0486
Epoch 3/100
36221/36221 [==============================] - 543s 15ms/step - loss: 3.5713 - accuracy: 0.0704 - val_loss: 3.4781 - val_accuracy: 0.0744
Epoch 4/100
36221/36221 [==============================] - 546s 15ms/step - loss: 3.3973 - accuracy: 0.0799 - val_loss: 3.3869 - val_accuracy: 0.0751
Epoch 5/100
36221/36221 [==============================] - 546s 15ms/step - loss: 3.2857 - accuracy: 0.0867 - val_loss: 3.4226 - val_accuracy: 0.0836
Epoch 6/100
36221/36221 [==============================] - 538s 15ms/step - loss: 3.2074 - accuracy: 0.0931 - val_loss: 3.2451 - val_accuracy: 0.0873
Epoch 7/100
36221/36221 [==============================] - 544s 15ms/step - loss: 3.1485 - accuracy:

In [ ]:
def plot_accuracy(history):
    
    plt.plot(history.history['accuracy'],label='train accuracy')
    plt.plot(history.history['val_accuracy'],label='validation accuracy')
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='best')
    plt.savefig('Accuracy_v1_model_inceptionv3')
    plt.show()
    
def plot_loss(history):
    
    plt.plot(history.history['loss'],label="train loss")
    plt.plot(history.history['val_loss'],label="validation loss")
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='best')
    plt.savefig('Loss_v1_model_inceptionv3')
    plt.show()
    
plot_accuracy(model_fit)
plot_loss(model_fit)

In [ ]:
model.save('/content/drive/MyDrive/Capstone/model_exercise.h5')
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('/content/drive/MyDrive/Capstone/model_exercise.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
from keras.models import model_from_json
# serialize model to json
json_model_exercise = model.to_json()
#save the model architecture to JSON file
with open('/content/drive/MyDrive/Capstone/exercise_model.json', 'w') as json_file:
    json_file.write(json_model_exercise)

In [ ]:
predict_x = model.predict(x_test) 
classes_x = np.argmax(predict_x,axis=1)
#y_pred_class = model.predict_classes(x_test)

y_pred = model.predict(x_test)
y_test_class = np.argmax(y_test, axis=1)
confusion_matrix = confusion_matrix(y_test_class, classes_x)

In [ ]:

report = classification_report(y_test_class, classes_x, output_dict=True, zero_division=0)
print(report)

# Extract the metrics
precision = report['macro avg']['precision']
recall = report['macro avg']['recall']
f1_score = report['macro avg']['f1-score']
support = report['macro avg']['support']
accuracy = report['accuracy']

print("accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1_score)
print("support" , support)

In [ ]:
#print(type(confusion_matrix(y_test_class, classes_x)))
#print(y_test_class)
#print(y_test)
print(len(y_test_class))
print(len(classes_x))
print(len(np.unique(y_test_class)))
print(len(np.unique(classes_x)))

In [ ]:
dict_activity = dict(enumerate(df_new['activity'].cat.categories))
df_new['activity_code'] = df_new['activity'].cat.codes
print(df_new['activity_code'])
print(dict_activity)
df_new['activity_reversed'] = df_new['activity_code'].map(dict_activity)
df_y_test_class = pd.DataFrame(y_test_class, columns = ['activity_class'])
df_y_test_class['activity_class_reversed'] = df_y_test_class['activity_class'].map(dict_activity)
print(df_y_test_class)

In [ ]:





df_cm = pd.DataFrame(confusion_matrix(y_test_class, classes_x), columns=np.unique(y_test_class), index=np.unique(y_test_class))
df_cm.index.name = 'Actual'
df_cm.columns.name = 'Predicted'


f, ax = plt.subplots(figsize=(500, 500))
cmap = sns.cubehelix_palette(light=1, as_cmap=True)

sns.heatmap(df_cm, cbar=False, annot=True, cmap=cmap, square=True, fmt='.0f',
            annot_kws={'size': 10})
plt.title('Actuals vs Predicted')
plt.show()